# Importar as bibliotecas necessárias

In [2]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from sentence_transformers import util
from transformers import T5Tokenizer, T5ForConditionalGeneration
import os
from rouge_score import rouge_scorer
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")

# Parâmetros

In [3]:
# Número máximo de respostas similares retornadas na busca semântica
top_k = 5  

# Similaridade mínima para considerar uma resposta relevante
# (garante que apenas textos realmente próximos sejam considerados)
similarity_threshold = 0.6  

# Limite mínimo para validar se a resposta está dentro do contexto esperado
# (descarta respostas que tenham pouca relação com a pergunta)
min_context_threshold = 0.3  

# Peso atribuído à similaridade entre o conteúdo do artigo e a pergunta
# (prioriza a correspondência com o corpo do texto)
content_similarity_weight = 0.6  

# Peso atribuído à similaridade entre o título do artigo e a pergunta
# (ajuda a dar relevância ao título na busca por respostas)
title_similarity_weight = 0.4  

# Definir limiares duplos para filtrar respostas inadequadas

# Limiar crítico: perguntas abaixo desse valor são consideradas fora do escopo
critical_threshold = 0.3  

# Similaridade mínima para considerar uma resposta da FAQ válida
# (usado na decisão final para recuperar um artigo)
similarity_threshold_faq = 0.55  

# Leitura dos dados

In [4]:
# Caminho do arquivo
file_path = r"C:\Users\Acer\Downloads\ProcessoSeletivo[RAG].xlsx"

# Carregar o CSV
df = pd.read_excel(file_path)

# Verificando as primeiras linhas
df.head()

,article_name,article_url,article_content
0,Como usar o Recuperador Automático de Vendas?,https://help.hotmart.com/pt-BR/article/como-us...,Como usar o Recuperador Automático de Vendas?\...
1,Como agendar uma publicação em comunidades do ...,https://help.hotmart.com/pt-BR/article/como-ag...,Como agendar uma publicação em comunidades do ...
2,Como configurar o pixel da Taboola,https://help.hotmart.com/pt-BR/article/como-co...,Como configurar o pixel da Taboola\n\n[Taboola...
3,Como navegar pelo app da Hotmart,https://help.hotmart.com/pt-BR/article/como-na...,Como navegar pelo app da Hotmart\n\nO app da H...
4,O que são parâmetros de campanha?,https://help.hotmart.com/pt-BR/article/o-que-s...,O que são parâmetros de campanha?\n\nOs parâme...


# Pré processamento

In [5]:
def clean_text(text, query):
    
    """Remove caracteres especiais, tags HTML, repetição da pergunta e elementos indesejados no início da resposta."""
    text = re.sub(r"<[^>]+>", "", text)  # Remove tags HTML
    text = re.sub(r"^[^\w]+", "", text)  # Remove caracteres especiais no início
    text = re.sub(r"\s+", " ", text).strip()  # Remove múltiplos espaços
    text = re.sub(r"={3,}", "", text).strip()  # Remove sequências de "="
    text = re.sub(r"^\?+\s*=\s*", "", text).strip()  # Remove "? ="

    # Removendo trechos redundantes no início da resposta
    query_clean = query.lower().strip("?").strip()
    text_clean = text.lower().strip()

    if text_clean.startswith(query_clean):
        text = text[len(query_clean):].strip()

    return text

In [6]:
def clean_response(response):
    """
    Remove o texto até o primeiro '?' para evitar repetições da pergunta na resposta.
    """
    response = response.strip()

    # Se houver "?" na resposta, cortamos tudo até ele
    if "?" in response:
        response = response.split("?", 1)[-1].strip()

    return response

In [8]:
# Carregar o modelo de embeddings
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Criar embeddings para os textos da FAQ
df["embeddings"] = df["article_content"].apply(lambda x: embedding_model.encode(x, normalize_embeddings=True))

# Converter para matriz numpy
embeddings_matrix = np.array(df["embeddings"].tolist())

# Verificando a dimensão dos embeddings
print("Dimensão dos embeddings:", embeddings_matrix.shape)

Dimensão dos embeddings: (477, 384)


In [9]:
# Obtém a dimensão dos embeddings
dimension = embeddings_matrix.shape[1]

# Criando índice FAISS usando a distância L2 (euclidiana)
index_cpu = faiss.IndexFlatL2(dimension)

# Adicionando os embeddings ao índice
index_cpu.add(embeddings_matrix)

# Verificando se os embeddings foram indexados corretamente
print(f"Índice FAISS criado com {index_cpu.ntotal} embeddings.")

Índice FAISS criado com 477 embeddings.


# Modelagem - RAG

## Recuperação da informação

In [10]:
def search_faq(query, top_k, similarity_threshold, min_context_threshold, content_similarity_weight, title_similarity_weight, critical_threshold, similarity_threshold_faq):

    """
    Busca uma resposta no FAQ com base na similaridade semântica.
    Inclui um filtro para evitar respostas fora do contexto.
    """

    # Criar embedding para a consulta
    query_embedding = embedding_model.encode([query], normalize_embeddings=True)

    # Buscar no índice FAISS
    distances, indices = index_cpu.search(np.array(query_embedding), k=top_k)

    # Filtrar apenas índices válidos
    valid_results = [(idx, dist) for idx, dist in zip(indices[0], distances[0]) if idx != -1]

    if not valid_results:
        return None  # Retorna None para indicar que não encontrou resposta relevante

    # Comparar a similaridade entre a pergunta e as respostas recuperadas
    best_match = None
    highest_score = -1

    for idx, dist in valid_results:
        retrieved_text = df.iloc[idx]["article_content"]
        article_title = df.iloc[idx]["article_name"]

        # Similaridade entre a pergunta e o título do artigo
        title_similarity = util.cos_sim(
                                            embedding_model.encode(query, convert_to_tensor=True),
                                            embedding_model.encode(article_title, convert_to_tensor=True)
                                        ).item()

        # Similaridade entre a pergunta e o conteúdo recuperado
        content_similarity = util.cos_sim(
                                                embedding_model.encode(query, convert_to_tensor=True),
                                                embedding_model.encode(retrieved_text, convert_to_tensor=True)
                                          ).item()

        # Score final priorizando título + conteúdo
        final_score = (content_similarity_weight * content_similarity) + (title_similarity_weight * title_similarity)

        # Definir limiares duplos
        critical_threshold = critical_threshold  # Perguntas abaixo disso são consideradas fora do escopo
        similarity_threshold = similarity_threshold  # Limiar para considerar resposta da FAQ válida

        # Adicionar verificação para evitar respostas fora de contexto
        if final_score > highest_score and final_score > similarity_threshold:
            highest_score = final_score
            best_match = idx

    # Se nenhuma resposta atingiu um mínimo aceitável, retorna "fora do escopo"
    if highest_score < critical_threshold:
        return "Desculpe, essa pergunta parece estar fora do escopo da Hotmart."

    # Se encontrou uma resposta válida, retorna
    if best_match is not None and highest_score > min_context_threshold:
        return clean_text(df.iloc[best_match]["article_content"], query)
    
    # Se nenhuma resposta for válida, retorna None (indicando que devemos gerar via IA)
    return None


## Generativa

In [11]:
 # Desativa avisos sobre links simbólicos no Hugging Face
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"


# Carregar modelo de geração de texto (FLAN-T5 Small, otimizado para eficiência)
model_name = "google/flan-t5-small"

# Inicializar o tokenizador e o modelo pré-treinado
tokenizer = T5Tokenizer.from_pretrained(model_name)
generator_model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [12]:
def generate_response(query, retrieved_text):

    """
    Gera uma resposta usando um modelo T5 baseado no texto recuperado.
    """

    # Construção do prompt, fornecendo o contexto e a pergunta ao modelo
    prompt = (
                f"Baseado no seguinte contexto, responda a pergunta de forma curta e objetiva:\n"
                f"Contexto: {retrieved_text}\n"
                f"Pergunta: {query}\n"
                f"Resposta:"
            )
    
    # Tokenização do prompt para entrada no modelo T5
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    
    # Geração da resposta usando o modelo T5 (máximo de 256 tokens na saída)
    output = generator_model.generate(**inputs, max_length=256, num_return_sequences=1)

    # Decodificação do resultado gerado para texto legível
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response

In [13]:
def process_response(query):
    
    """ 
    Pipeline para buscar, limpar e formatar a resposta.
    Primeiro, tenta recuperar a resposta do FAQ. Caso não encontre, aciona a geração via IA.
    """
    # Busca a resposta no FAQ usando a função de recuperação semântica (FAISS + embeddings)
    retrieved_response = search_faq(query, top_k, similarity_threshold, min_context_threshold, content_similarity_weight, title_similarity_weight, critical_threshold, similarity_threshold_faq)  # Busca no FAQ

    # Se uma resposta válida for encontrada no FAQ, retorna diretamente
    if retrieved_response:
        return retrieved_response  # Se encontrou resposta válida no FAQ, retorna
    
    # Caso nenhuma resposta relevante seja encontrada, aciona a geração de texto com IA
    return generate_response(query, "Infelizmente, não há informações disponíveis no FAQ.")

## Testes

### Perguntas que temos na base

In [23]:
# Lista de perguntas de teste usadas para avaliar a recuperação de respostas do FAQ
test_questions_recuperacao_pura = [
                                        "Como reprocessar meu pagamento?",
                                        "Como acessar o Relatório de Download de Notas Fiscais?",
                                        "Como encerrar minhas vendas e desabilitar a página de pagamento?",
                                        "Como liberar o cadastro gratuito do meu produto no Hotmart Club?",
                                        "Por que recebi um e-mail de verificação depois de fazer uma solicitação na Central de Ajuda?"
                                    ]

# Criação de um dicionário onde as perguntas são as chaves e as respostas esperadas (FAQ) são os valores
test_questions_generation = {
                                # Verifica se a pergunta está contida no título dos artigos do FAQ
                                q: df[df['article_name'].str.contains(q, case=False, na=False)]['clean_article_content'].values[0] 
                                if df['article_name'].str.contains(q, case=False, na=False).any() 
                                else "Resposta não encontrada."
                                for q in test_questions_recuperacao_pura
                            }

# Ajuste para verificar se há correspondência exata entre as perguntas e os títulos dos artigos
for q in test_questions_recuperacao_pura:
    matched_rows = df[df['article_name'].str.lower() == q.lower()]# Normaliza para evitar erros de case-sensitive
    
    if not matched_rows.empty:
        # Se encontrar uma correspondência exata no título do artigo, usa a resposta desse artigo
        test_questions_generation[q] = matched_rows.iloc[0]['clean_article_content']
    else:
        # Caso não haja correspondência exata, mantém como "Resposta não encontrada."
        test_questions_generation[q] = "Resposta não encontrada."

### Perguntas que não temos na base

In [52]:
test_questions_recuperacao_pura = [    
                                        "Como posso modificar o método de pagamento da minha assinatura?",
                                        "Quais são os métodos de pagamento disponíveis na Hotmart?",
                                        "O que é o PIX da Hotmart e como ativá-lo?",
                                        "Como alterar o idioma na página de pagamento da Hotmart?",
                                        "Quanto a Hotmart cobra de taxas nas vendas?",
                                    ]

test_questions_ajuste_generativo = [    
                                    "Posso pagar com dois cartões de crédito?",
                                    "Quais são as taxas para produtores na Hotmart?",
                                    "Como funciona o parcelamento de pagamentos?",
                                    "Posso oferecer reembolsos na Hotmart?",
                                    "Como configurar descontos para meus clientes na Hotmart?",
                                ]

test_questions_geracao_pura = [    
                                    "A Hotmart aceita criptomoedas como forma de pagamento?",
                                    "Posso pagar um curso na Hotmart com Apple Pay?",
                                    "Existe algum benefício para quem paga à vista?",
                                    "A Hotmart oferece suporte para pagamentos via boleto parcelado?",
                                    "Quais são as opções para receber pagamentos em diferentes moedas?"
                                ]

test_questions_geracao_pura_fora_do_contexto = [    
                                                    "Qual a distância da Terra até a Lua?",
                                                    "Quem foi o primeiro presidente do Brasil?",
                                                    "Quem escreveu Dom Quixote?",
                                                    "Qual é a capital da Islândia?",
                                                    "Como posso aprender a tocar violão do zero?"
                                                ]

In [ ]:
# Iterar sobre todas as perguntas de teste
for query in test_questions_recuperacao_pura:
    # Recuperar e processar resposta

    final_response = process_response(query)
    final_response = clean_response(final_response)

    # Exibir resposta corrigida
    print("="*80)
    print("Pergunta:", query)
    print("Resposta:", final_response)
    print("="*80, "\n")

Pergunta: Como posso modificar o método de pagamento da minha assinatura?
Resposta: = Alterar o método de pagamento da sua assinatura para evitar imprevistos de acesso é muito simples. Com apenas cinco passos, você terá a facilidade de ajustar a configuração de pagamento usada no momento da compra, incluindo a flexibilidade de [alterar a data de cobrança conforme sua necessidade](https://help.hotmart.com/pt-br/article/14718188112781). Isso proporciona a você maior controle sobre o processo, garantindo uma melhor experiência na gestão da sua assinatura. Neste artigo, vamos te ensinar a alterar o método de pagamento conforme a sua necessidade. Para isso, basta seguir os passos indicados abaixo: 1. Acesse . 2. Após fazer o login, selecione Minhas compras no menu à esquerda e clique no produto pelo qual deseja trocar seu método de pagamento. 3. Os detalhes da sua compra serão exibidos e você poderá selecionar a opção Configurar pagamento (ou Regularizar pagamentos pendentes, caso o pagamen

In [29]:
# Iterar sobre todas as perguntas de teste
for query in test_questions_ajuste_generativo:
    # Recuperar e processar resposta
    final_response = process_response(query)
    final_response = clean_response(final_response)

    # Exibir resposta corrigida
    print("="*80)
    print("Pergunta:", query)
    print("Resposta:", final_response)
    print("="*80, "\n")

Pergunta: Posso pagar com dois cartões de crédito?
Resposta: Ao efetuar compras, pode ocorrer dos Compradores enfrentarem alguns problemas com limites do cartão de crédito. Isso pode acontecer tanto por falta de limite num único cartão ou, em outros casos, pelo desejo de não comprometer todo o saldo que há nele. Dessa forma, a melhor maneira de evitar a perda dessas vendas é habilitar a opção de pagamento com dois cartões de crédito em seu checkout (página de pagamento). Quer saber como? Para ativar o pagamento com dois cartões de créditos diferentes, basta seguir o passo a passo abaixo: 1. Faça login na plataforma por meio do link ; 2. Vá em Ferramentas; 3. Clique em Configurações de pagamento; 4. Selecione o produto. Nele, ative a opção desejada; 5. Ative a opção 2 Cartões de Crédito; 6. Por fim, clique em Salvar. Atenção: para vendas no México e Colômbia, ainda não é possível ativar o pagamento com dois cartões de crédito.

Pergunta: Quais são as taxas para produtores na Hotmart?
Re

In [30]:
# Iterar sobre todas as perguntas de teste
for query in test_questions_geracao_pura:
    # Recuperar e processar resposta
    final_response = process_response(query)
    final_response = clean_response(final_response)

    # Exibir resposta corrigida
    print("="*80)
    print("Pergunta:", query)
    print("Resposta:", final_response)
    print("="*80, "\n")

Pergunta: A Hotmart aceita criptomoedas como forma de pagamento?
Resposta: A conta de pagamento Hotmart é gerada de forma automática, para usuários brasileiros, assim que você realiza sua primeira venda. Diferente da conta bancária que você cadastra para realizar os saques, a conta de pagamento Hotmart é criada para acessar seus ganhos de forma segura e eficaz, cujo valor é mantido em uma conta do Banco Central do Brasil (Bacen). Isso significa que seu dinheiro está em segurança no Bacen, sendo você a única pessoa com acesso à sua conta. A Hotmart não tem permissão para acessar ou movimentar seu saldo sem sua autorização. Para mais detalhes, consulte nossa [Política Geral de Pagamentos](https://hotmart.com/pt-br/legal/politicas-de-pagamento). A abertura dessa conta cumpre uma [determinação do Bacen](https://help.hotmart.com/pt-BR/article/o-que-e-e-como-funciona-uma-sociedade-de-credito-direto-/15854959057037), que inclui a obrigação de atualizar anualmente as informações da mesma. Para

In [31]:
# Iterar sobre todas as perguntas de teste
for query in test_questions_geracao_pura_fora_do_contexto:
    # Recuperar e processar resposta

    final_response = process_response(query)
    final_response = clean_response(final_response)

    # Exibir resposta corrigida
    print("="*80)
    print("Pergunta:", query)
    print("Resposta:", final_response)
    print("="*80, "\n")

Pergunta: Qual a distância da Terra até a Lua?
Resposta: Desculpe, essa pergunta parece estar fora do escopo da Hotmart.

Pergunta: Quem foi o primeiro presidente do Brasil?
Resposta: Desculpe, essa pergunta parece estar fora do escopo da Hotmart.

Pergunta: Quem escreveu Dom Quixote?
Resposta: Desculpe, essa pergunta parece estar fora do escopo da Hotmart.

Pergunta: Qual é a capital da Islândia?
Resposta: Desculpe, essa pergunta parece estar fora do escopo da Hotmart.

Pergunta: Como posso aprender a tocar violão do zero?
Resposta: Desculpe, essa pergunta parece estar fora do escopo da Hotmart.



## Validação do modelo

In [32]:
# Criar um dicionário onde o título do artigo ('article_name') é a chave e o conteúdo limpo do artigo ('clean_article_content') é o valor.
# Isso permite uma busca eficiente das respostas da FAQ com base no título do artigo.
respostas_faq = {row['article_name']: row['clean_article_content'] for _, row in df.iterrows()}

# 🔹 Criar um dicionário consolidado para os testes de geração de respostas
test_questions_generation = {
                                # Para cada conjunto de perguntas de teste, busca a resposta correspondente na base de conhecimento (FAQ)
                                # Se a pergunta não existir no dicionário 'respostas_faq', retorna "Resposta não encontrada."
                                **{q: respostas_faq.get(q, "Resposta não encontrada.") for q in test_questions_recuperacao_pura},
                                **{q: respostas_faq.get(q, "Resposta não encontrada.") for q in test_questions_ajuste_generativo},
                                **{q: respostas_faq.get(q, "Resposta não encontrada.") for q in test_questions_geracao_pura}
                            }

In [55]:
def evaluate_generation():
    
    # Listas para armazenar as métricas BLEU e ROUGE de cada resposta gerada
    bleu_scores = []
    rouge_scores = []
    
    # Itera sobre todas as perguntas de teste e suas respostas esperadas
    for query, expected_response in test_questions_generation.items():
        generated_response = process_response(query) # Obtém a resposta gerada pelo modelo

        # Verifica se a resposta indica que a pergunta está fora do escopo
        if "Desculpe, essa pergunta parece estar fora do escopo da Hotmart." in generated_response:
            continue  # Ignora perguntas fora do contexto ao calcular métricas

        # Calcula a métrica BLEU (mede similaridade entre frases)
        smoothing = SmoothingFunction().method1 # Suavização para evitar penalizações severas
        bleu = sentence_bleu([expected_response.split()], generated_response.split(), 
                            weights=(0.5, 0.5, 0, 0), # Peso para medir bigramas (considera contexto)
                            smoothing_function=smoothing)
        bleu_scores.append(bleu)

        # Calcula a métrica ROUGE-L (mede cobertura de palavras relevantes)
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)# Usa stemming para generalização
        rouge = scorer.score(expected_response, generated_response)
        rouge_scores.append(rouge['rougeL'].fmeasure) # Usa ROUGE-L F1-score como métrica principal

    # Exibir apenas métricas finais
    print("\n*MÉDIAS GERAIS:*")
    if bleu_scores and rouge_scores:
        print(f"BLEU Médio: {np.mean(bleu_scores):.4f}") # Exibe média de BLEU para todas as respostas
        print(f"ROUGE-L Médio: {np.mean(rouge_scores):.4f}") # Exibe média de ROUGE-L para todas as respostas
    else:
        print("⚠️ Nenhuma métrica foi computada, pois todas as perguntas foram consideradas fora do escopo.")

# Rodar a avaliação
evaluate_generation()


*MÉDIAS GERAIS:*
BLEU Médio: 0.5647
ROUGE-L Médio: 0.5847
